In [18]:
import pandas
import numpy
import torch

#Mount del drive per accedere al dataset


In [15]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
percorso =  "/content/drive/MyDrive/Progetto_laboratorio"

Mounted at /content/drive
